In [20]:
import sys
import os
import re
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Sentiment analysis of the comments

In [11]:
tweets_excel_file = 'AlertMPK comments.xlsx'
comments_sheet = pd.read_excel(tweets_excel_file, sheet_name='mainSheet')

In [12]:
comments_sheet

,id,post_id,cleaned_comment_without_username,label,Unnamed: 4
0,5dd483a6804e3d1bf0e8c321,5dd483a3804e3d1bf0e8c320,Autobusy na suchej jeżdżą normalnie? Czy przez...,0.0,0
1,5dd483a8804e3d1bf0e8c322,5dd483a3804e3d1bf0e8c320,Ponieważ,0.0,unknown
2,5dd4850e804e3d1bf0e8c3ea,5dd4850d804e3d1bf0e8c3e9,"Jest 5 Inwestycji, a jeszcze dwie nam dochodzą...",0.0,0
3,5dd48510804e3d1bf0e8c3eb,5dd4850d804e3d1bf0e8c3e9,"Bocian Super, czyli z Joannitów już niczym się...",0.0,-1
4,5dd4851b804e3d1bf0e8c3ed,5dd4851a804e3d1bf0e8c3ec,A jak to wpłynie na opóźnienia 118? Czy zostan...,-1.0,-1
...,...,...,...,...,...
3865,5dda5794073b739b4a9aa6fe,5dda5783073b739b4a9aa6fd,A u was klasycznie - wszystkie plagi tego świa...,NaN,NaN
3866,5dda579d073b739b4a9aa6ff,5dda5783073b739b4a9aa6fd,Już 3 awaria dziś dobry tydzień się zapowiada,NaN,NaN
3867,5dda57a4073b739b4a9aa700,5dda5783073b739b4a9aa6fd,"chyba nie bardzo wróciły na swoje trasy, bo pr...",NaN,NaN
3868,5dda57b3073b739b4a9aa701,5dda5783073b739b4a9aa6fd,"mam nadzieje,ze jestes w Brzegu.",NaN,NaN


In [14]:
ids = list(comments_sheet['id'])
comments = list(comments_sheet['cleaned_comment_without_username'])

Save comments and ids in seperatated files

-ids are remained to preserve mapping to the post and commentaries

In [18]:
with open("data/comments", 'x') as write_file:
    for comment in comments:
        write_file.write(str(comment) + os.linesep + os.linesep)

In [53]:
with open("data/ids", 'x') as write_file:
    for id in ids:
        write_file.write(str(id) + os.linesep)

Morpheus divides opinions into sentences, so cmc tagger also keeps this structure

Below we pull out the full sentences from xml structure

In [43]:
input_file = "data/comments-morf.tagged"
opinions_file = "data/comments"

out_file = "data/sentences-cmc"

root = ET.parse(input_file).getroot()

with open(out_file, 'w') as write_file:
    for chunk in root:
        skip_space = True
        for sentence in chunk:
            for token in sentence:
                if token.tag == 'tok':
                    if skip_space is False:
                        write_file.write(' ')
                    if skip_space is True:
                        skip_space = False
                    orth = token.find('orth')
                    write_file.write(orth.text)
                if token.tag == 'ns':
                    skip_space = True
            write_file.write(os.linesep)

Next, we take base form of the words except those types we know we would not use it to get text sentiment, here: interpunction and numerical values

and we place in the next file where every line is a sentence (with words in their basic forms)

In [46]:
filtered_file = "data/cmc-base"

root = ET.parse(input_file).getroot()

with open(base_form_sentences_file, 'w') as write_file:
    for chunk in root:
        for sentence in chunk:
            for token in sentence:
                if token.tag == 'tok':
                    lex = token.find('lex')
                    base_word = lex.find('base')
                    orth = token.find('orth')
                    ctag = lex.find('ctag')
                    if ctag.text.split(":", 1)[0] not in ['interp', 'num']:
                        write_file.write(base_word.text + ' ')
            write_file.write(os.linesep)

Using full opinions, full sentences and sentences with words modified to their basic form, we recreate opinions with modfied words

In [47]:
with open(opinions_file) as f:
    opinions = f.read().splitlines()
    
with open(out_file) as f:
    sentences = f.read().splitlines()

with open(filtered_file) as f:
    filtered_words = f.read().splitlines()

opinions = list(filter(lambda o: bool(o.strip()), opinions))

In [49]:
opinion_index = 0
sentence_index = 0
op_sent_nb = [0] * len(opinions)
opinions_filtered = [''] * len(opinions)
while opinion_index < len(opinions) and sentence_index < len(sentences):
    if sentences[sentence_index] in opinions[opinion_index]:
        op_sent_nb[opinion_index] = op_sent_nb[opinion_index] + 1
        if filtered_words[sentence_index].strip():
            opinions_filtered[opinion_index] = opinions_filtered[opinion_index] + filtered_words[sentence_index]
        sentence_index = sentence_index + 1
    else:
        opinion_index = opinion_index + 1    

In [50]:
output_path = "data/base-opinions-cmc"

In [51]:
with open(output_path, 'w') as write_file:
    for op in opinions_filtered:
        write_file.write(op + os.linesep)

In [57]:
with open("data/base-opinions-cmc") as f:
    base_comments = f.read().splitlines()
    
with open("data/ids") as f:
    ids = f.read().splitlines()

In [56]:
len(base_comments)

3870

In [61]:
base_df = pd.DataFrame(
    {
        'id': ids,
        'comment': base_comments
    })

In [62]:
base_df

,id,comment
0,5dd483a6804e3d1bf0e8c321,autobus na suchy jeździć normalnie czy przez t...
1,5dd483a8804e3d1bf0e8c322,ponieważ
2,5dd4850e804e3d1bf0e8c3ea,być 5 inwestycja a jeszcze my dochodzić zarąbi...
3,5dd48510804e3d1bf0e8c3eb,bocian super czyli z joannita już nic się nie ...
4,5dd4851b804e3d1bf0e8c3ed,a jak to wpłynąć na opóźnienie czy zostać wpro...
...,...,...
3865,5dda5794073b739b4a9aa6fe,a u wy klasycznie wszystek plaga ten świat w j...
3866,5dda579d073b739b4a9aa6ff,już 3 awaria dziś dobry tydzień się zapowiadać
3867,5dda57a4073b739b4a9aa700,chyba nie bardzo wrócić na swój trasa bo przez...
3868,5dda57b3073b739b4a9aa701,mieć Nadzieja z jestes w brzeg


The dictionary comes from http://clip.ipipan.waw.pl/LRT , exactly from http://zil.ipipan.waw.pl/SlownikWydzwieku

In [65]:
sentiment_dictionary = pd.read_csv('slownikWydzwieku01.csv', sep="\t", header=None)

In [74]:
dct = list(sentiment_dictionary[0])

In [82]:
def get_word_sentiment(word):
    if word in dct:
        return float(sentiment_dictionary.loc[sentiment_dictionary[0] == word, 5])
    else: 
        return 0.

In [106]:
def get_word_sentiment_in_text(text):
    words = text.strip().split(' ')
    return [(word, get_word_sentiment(word)) for word in words]

In [107]:
base_df.loc[4, 'comment']

'a jak to wpłynąć na opóźnienie czy zostać wprowadzić zmiana w rozkład jazda odnośnie godzina odjazd bo obecnie mieć zmienić trasa w związek z przebudowa ulica obornicki i choć on trasa w kierunek Sępolno Wojnowa znacznie się wydłużyć godzina odjazd nie zmienić się a autobus przyjeżdżać zawsze spóźnić teraz zostać skierować objazd przez zakorkować Swojczycką CO znowu wydłużyć trasa i zwiększyć i tak już w godzina szczyt zwłaszcza poranny gigantyczny opóźnienie '

In [108]:
get_word_sentiment_in_text(base_df.loc[4, 'comment'])

[('a', 0.0),
 ('jak', 6.1),
 ('to', 0.0),
 ('wpłynąć', 0.0),
 ('na', 0.0),
 ('opóźnienie', 0.0),
 ('czy', 0.0),
 ('zostać', 4.1),
 ('wprowadzić', 0.0),
 ('zmiana', 0.4),
 ('w', 0.0),
 ('rozkład', 0.0),
 ('jazda', 4.0),
 ('odnośnie', 0.0),
 ('godzina', -0.5),
 ('odjazd', 0.5),
 ('bo', 0.0),
 ('obecnie', -3.5),
 ('mieć', 4.3),
 ('zmienić', 4.1),
 ('trasa', 0.6),
 ('w', 0.0),
 ('związek', -0.4),
 ('z', 0.0),
 ('przebudowa', -0.7),
 ('ulica', 0.0),
 ('obornicki', 0.0),
 ('i', 0.0),
 ('choć', 0.0),
 ('on', 0.0),
 ('trasa', 0.6),
 ('w', 0.0),
 ('kierunek', 0.0),
 ('Sępolno', 0.0),
 ('Wojnowa', 0.0),
 ('znacznie', 0.4),
 ('się', 0.0),
 ('wydłużyć', -2.0),
 ('godzina', -0.5),
 ('odjazd', 0.5),
 ('nie', 0.0),
 ('zmienić', 4.1),
 ('się', 0.0),
 ('a', 0.0),
 ('autobus', 0.0),
 ('przyjeżdżać', 0.0),
 ('zawsze', 5.7),
 ('spóźnić', 0.0),
 ('teraz', 0.0),
 ('zostać', 4.1),
 ('skierować', -1.9),
 ('objazd', 0.0),
 ('przez', 0.0),
 ('zakorkować', 0.0),
 ('Swojczycką', 0.0),
 ('CO', 0.0),
 ('znowu', 3.8

#### two approaches to calculate sentiment

In [114]:
# sum sentiment
def get_text_sentiment(text):
    words = text.strip().split(' ')
    return sum([get_word_sentiment(word) for word in words])

In [115]:
get_text_sentiment(base_df.loc[4, 'comment'])

27.4

In [116]:
# mean sentiment
def get_text_sentiment2(text):
    words = text.strip().split(' ')
    words_sentiment = [get_word_sentiment(word) for word in words]
    return sum(words_sentiment)/len(words_sentiment)

In [117]:
get_text_sentiment2(base_df.loc[4, 'comment'])

0.38591549295774646

In [120]:
base_df['sum_sentiment'] = base_df['comment'].apply(get_text_sentiment)

In [121]:
base_df['mean_sentiment'] = base_df['comment'].apply(get_text_sentiment2)

In [125]:
base_df.sort_values(by='sum_sentiment')[:10]

,id,comment,sum_sentiment,mean_sentiment
3320,5dd710e9bbb2205e91175145,straż Miajska zostać wezwać natychmiast po inf...,-15.7,-0.541379
3423,5dd724ddbbb2205e911753e1,by si zalorkowaly ostatnio byly test system IT...,-13.8,-0.492857
1131,5dd60adc093b64288e1c1f69,Stankiewicz pogorszenie w krótki czas to ja by...,-11.8,-0.218519
3018,5dd7042fbbb2205e91174f13,pęknąć przyczłapać bulgulatora w lewy silnik,-11.2,-1.866667
3591,5dd85c6a4322651b41f6ea63,gazeta wrocławski z tramwaj wypaść silnik http...,-11.1,-1.585714
1440,5dd67375aff40d93edbfbfb5,państwo się śmiać a to nie być taki oczywisty ...,-10.7,-0.218367
152,5dd52c92bea9ce7c8811b565,super dzięki wydział transport za reakcja na z...,-10.7,-1.337500
3309,5dd710cdbbb2205e91175139,to faktycznie trzeba szampan otworzyć ja siebi...,-10.5,-0.617647
1054,5dd5fc4c093b64288e1c1e37,MPK być tylko przewoznikiem wiec chyba logiczn...,-10.4,-0.520000
1923,5dd6db07bbb2205e91174394,tak a policja na Gądowiance zablokować minuta ...,-9.8,-0.980000


In [127]:
base_df.sort_values(by='sum_sentiment').iloc[0]['comment']

'straż Miajska zostać wezwać natychmiast po informacja od kierowca o brak przejazd ostatecznie i tak przyjechać policja który wezwać laweta i wywieźć auto następny raz nie oceniać w ciemno '

In [129]:
base_df.sort_values(by='sum_sentiment', ascending=False)

,id,comment,sum_sentiment,mean_sentiment
2430,5dd6ee52bbb2205e91174a34,Warszawa być po prosty nastawić na elastyczny ...,151.2,0.465231
1394,5dd67208aff40d93edbfbf85,różny przystanek to przykład to że kierowca ni...,149.4,1.114925
465,5dd592776af02b29ffa78df2,pełny zgoda pani Dominika dzisiaj dla większoś...,147.9,1.071739
400,5dd56cafa1ab399b2bea05d4,Linia C nie być dla każdy pasażer a zapoznać s...,142.3,1.111719
5,5dd485b2804e3d1bf0e8c42b,wszystko super tylko szkoda że zablokować być ...,124.7,1.022131
...,...,...,...,...
3591,5dd85c6a4322651b41f6ea63,gazeta wrocławski z tramwaj wypaść silnik http...,-11.1,-1.585714
3018,5dd7042fbbb2205e91174f13,pęknąć przyczłapać bulgulatora w lewy silnik,-11.2,-1.866667
1131,5dd60adc093b64288e1c1f69,Stankiewicz pogorszenie w krótki czas to ja by...,-11.8,-0.218519
3423,5dd724ddbbb2205e911753e1,by si zalorkowaly ostatnio byly test system IT...,-13.8,-0.492857


##  Use polyglot to get polarity

In [144]:
from polyglot.text import Text
import preprocessor as preprocessor

In [150]:
def get_polarity(phrase):
    try:
        phrase = preprocessor.clean(phrase) 
        return Text(phrase).polarity
    except:
        return 0

In [155]:
get_polarity(list(comments_sheet.loc[comments_sheet['id'] == com_id, 'cleaned_comment_without_username'])[0])

0.14285714285714285

In [157]:
base_df['polarity'] = base_df['comment'].apply(get_polarity)

In [158]:
base_df.to_excel('results.xlsx')

In [159]:
base_df['polarity_nonbase'] = comments_sheet['cleaned_comment_without_username'].apply(get_polarity)

In [161]:
base_df['comment_nonbase'] = comments_sheet['cleaned_comment_without_username']

In [162]:
base_df.to_excel('results.xlsx')

## Fasttext model

In [171]:
import fasttext

In [168]:
c_df = comments_sheet[comments_sheet['label'].notnull()]

array([ 0., -1.,  1.])

In [169]:
label_prefix = '__label__'
with open('labeled_comments', 'w') as file:
    for index, row in c_df.iterrows():
        file.write(label_prefix + str(int(row['label'])) + ' ')
        file.write(row['cleaned_comment_without_username'].replace('\n', ' '))
        file.write('\n')

In [172]:
model = fasttext.train_supervised(input="labeled_comments", epoch=25, lr=1.0, wordNgrams=2)

In [183]:
def make_prediction(text):
    prediction = model.predict(str(text))
    return prediction[0][0]

In [184]:
def get_prediction_prob(text):
    prediction = model.predict(str(text))
    return prediction[1][0]

In [185]:
base_df['fasttext_prediction'] = comments_sheet['cleaned_comment_without_username'].apply(make_prediction)

In [186]:
base_df['fasttext_prob'] = comments_sheet['cleaned_comment_without_username'].apply(get_prediction_prob)

In [187]:
base_df.to_excel('results.xlsx')